# Verzeo - Major Project (ML-JULY-B1)

### Done by : Sanjay Marreddi  
### Email Id  : sanjay.marreddi.19041@iitgoa.ac.in
    

#### First let us import the required Libraries

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer,PorterStemmer

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

#### Let us read the given Dataset

In [ ]:
information = pd.read_excel('information.xlsx')

## Exploratory data analysis and Data Cleaning

In [ ]:
information.head()

In [ ]:
information.shape

In [ ]:
information.columns.tolist()

In [ ]:
information.describe()

In [ ]:
information.info()

In [ ]:
information.isnull().sum() # Checking the Missing Values in each Column of the DataFrame

In [ ]:
information.corr()

In [ ]:
# HaetMap of the Correlation Matrix
sns.heatmap(information.corr())

In [ ]:
information.description

In [ ]:
information.name

In [ ]:
information.text

###### Feature Selection

In [ ]:
information =  information[["gender", "name","description", "gender:confidence","text"]]

###### Dropping the rows with NaN 

In [ ]:
information = information.dropna()

In [ ]:
information.shape

#### Data Cleaning of "description" column

In [ ]:
from nltk.corpus import stopwords

In [ ]:
list_of_stopwords = list(stopwords.words('english'))

In [ ]:
information.shape

In [ ]:
information.columns

In [ ]:
# Taking only those rows which has type of description str. 
temp=[]
for i in range(information.shape[0]):
    temp.append(type(information['description'].values[i]) == str)

In [ ]:
information = information.loc[temp,:]

In [ ]:
stemmer = PorterStemmer()
words = stopwords.words("english")
# Data Cleaning using regex.
information['cleaned_description'] = information['description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
information.head()

#### Data Cleaning of "text" column

In [ ]:
information.columns

In [ ]:
stemmer = PorterStemmer()
words = stopwords.words("english")
information['cleaned_text'] = information['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
information.head()

In [ ]:
# Taking only those rows with full confidence in gender
information = information[information["gender:confidence"]==1]

In [ ]:
information.head()

In [ ]:
X_information =  information[["name","cleaned_text","cleaned_description"]]

y_information = information[["gender"]] # Target Variable

In [ ]:
print (X_information.shape)
X_information.head()

In [ ]:
print(X_information.shape)
y_information.head()

##### Label Encoding the gender column for ease of Calculation 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_information['gender_encoded'] = le.fit_transform(y_information["gender"]) 

In [ ]:
y_information = y_information[["gender_encoded"]]

In [ ]:
print (X_information.shape)
X_information.head()

In [ ]:
print(X_information.shape)
y_information.head()

# Ensemble Machine Learning Modelling

## 1. Classification using Naive Bayes Algorithm 

In [ ]:
print (X_information.shape)
X_information.head()

In [ ]:
print(y_information.shape)
y_information.head()

In [ ]:
Train_X_Nb, Test_X_Nb, Train_Y_Nb, Test_Y_Nb = train_test_split(X_information['cleaned_description'],y_information['gender_encoded'],test_size = 0.01, random_state=1)

In [ ]:
# Doing Label Encoding for both Training and Testing Sets.
Encoder = LabelEncoder()
Train_Y_Nb = Encoder.fit_transform(Train_Y_Nb)
Test_Y_Nb = Encoder.fit_transform(Test_Y_Nb)

In [ ]:
# Transforming text to feature vectors that can be used as input to estimator.
T_vect = TfidfVectorizer(max_features=5000)
T_vect.fit(X_information['cleaned_description'])

In [ ]:
# Transforming the Train and Test sets.
Train_X_Nb_T = T_vect.transform(Train_X_Nb)
Test_X_Nb_T = T_vect.transform(Test_X_Nb)

In [ ]:
# fit the training dataset on the NB classifier.
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Nb_T,Train_Y_Nb)

In [ ]:
# predict the labels on validation dataset.
predictions_Nb = Naive.predict(Test_X_Nb_T)

# Use accuracy_score function to get the accuracy.
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_Nb, Test_Y_Nb)*100)

## 2. Classification using Support Vector Machine Algorithm 

In [ ]:
print (X_information.shape)
X_information.head()

In [ ]:
print (y_information.shape)
y_information.head()

In [ ]:
Train_X_Svm, Test_X_Svm, Train_Y_Svm, Test_Y_Svm = train_test_split(X_information['cleaned_description'],y_information['gender_encoded'],test_size = 0.01, random_state=10)

In [ ]:
# Doing Label Encoding for both Training and Testing Sets.
Encoder = LabelEncoder()
Train_Y_Svm = Encoder.fit_transform(Train_Y_Svm)
Test_Y_Svm = Encoder.fit_transform(Test_Y_Svm)

In [ ]:
# Transforming text to feature vectors that can be used as input to estimator.
T_vect = TfidfVectorizer(max_features=5000)
T_vect.fit(X_information['cleaned_description'])

In [ ]:
# Transforming the Train and Test sets.
Train_X_Svm_T = T_vect.transform(Train_X_Svm)
Test_X_Svm_T = T_vect.transform(Test_X_Svm)

In [ ]:
# fit the training dataset on the classifier
SVM = svm.SVC(C=2.0, kernel='linear', degree=2, gamma='auto')
SVM.fit(Train_X_Svm_T,Train_Y_Svm)

In [ ]:
# predict the labels on validation dataset
predictions_Svm = SVM.predict(Test_X_Svm_T)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_Svm, Test_Y_Svm)*100)

## 3. Classification using Logistic Regression Algorithm 

In [ ]:
print (X_information.shape)
X_information.head()

In [ ]:
print(y_information.shape)
y_information.head()

In [ ]:
Train_X_Lr, Test_X_Lr, Train_Y_Lr, Test_Y_Lr = train_test_split(X_information['cleaned_description'],y_information['gender_encoded'],test_size = 0.01, random_state=22)

In [ ]:
# Doing Label Encoding for both Training and Testing Sets.
Encoder = LabelEncoder()
Train_Y_Lr = Encoder.fit_transform(Train_Y_Lr)
Test_Y_Lr = Encoder.fit_transform(Test_Y_Lr)

In [ ]:
# Transforming text to feature vectors that can be used as input to estimator.
T_vect = TfidfVectorizer(max_features=5000)
T_vect.fit(X_information['cleaned_description'])

In [ ]:
# Transforming the Train and Test sets.
Train_X_Lr_T = T_vect.transform(Train_X_Lr)
Test_X_Lr_T = T_vect.transform(Test_X_Lr)

In [ ]:
# fit the training dataset on the classifier
lr = LogisticRegression(max_iter = 1000)
lr.fit(Train_X_Lr_T,Train_Y_Lr)

In [ ]:
# predict the labels on validation dataset
predictions_Lr = lr.predict(Test_X_Lr_T)

# Use accuracy_score function to get the accuracy
print("LR Accuracy Score -> ",accuracy_score(predictions_Lr, Test_Y_Lr)*100)

### Ensemble Machine Learning Modelling -> Choosing best Algo based on Accuracy

###### From above Models, Accuracy obtained through

1. Naive Bayes Algorithm           is **71.42857142857143 %**

2. Support Vector Machine Algorithm is **68.0672268907563 %**

3. Logistic Regression Algorithm is  **70.58823529411765 %**




###### So, For the given data set and the used hyperparameters, Based on the accuracy score , We can say **Naive Bias Algorithm is best** in this case.

# Questions on DataSet 

###### 1.What are the most common emotions/words used by Males and Females?

In [ ]:
information = pd.read_excel('information.xlsx')

information =  information[["gender", "name","description", "gender:confidence","text"]]

information = information.dropna()

# Taking only those rows which has type of description str. 
temp=[]
for i in range(information.shape[0]):
    temp.append(type(information['description'].values[i]) == str)
    
information = information.loc[temp,:]

stemmer = PorterStemmer()
words = stopwords.words("english")
# Data Cleaning using regex.
information['cleaned_description'] = information['description'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())


stemmer = PorterStemmer()
words = stopwords.words("english")
information['cleaned_text'] = information['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

information = information[information["gender:confidence"]==1]


X_information =  information[["gender", "name","cleaned_text","cleaned_description"]]


In [ ]:
X_information.gender.value_counts()

In [ ]:
X_info = X_information.loc[ (X_information.gender == "female" ) | (X_information.gender == "male" ) ] 

In [ ]:
X_info